In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from libary import *
from recurrent import *
from ARMA import *

In [2]:
# Used data
num_of_features = 2

In [3]:
# Import data
Data = pd.read_csv('Data_1min.csv', dtype=str)

# Select frequency
frequencies = ['min','5min','15min','60min','D']
frequencies_number_of_samples = np.multiply([390,78,26,6,1],252)
frequency_index = 0
Data = resample_data(Data,frequencies[frequency_index])

# Create datasets
dates = Data['Date']
Data.drop('Date', inplace=True, axis=1)

Data = Data.astype('float64')
prices = Data

# Log return
Returns = log_return(Data).to_numpy()

# Visualize data
#visualise_data(Data, Returns)

# Create study peridos
number_of_study_periods = np.floor(Returns.shape[0]/frequencies_number_of_samples[frequency_index]).astype(int)-1
study_periods = np.zeros((2,number_of_study_periods, frequencies_number_of_samples[frequency_index]*2))
for i in range(number_of_study_periods):
    study_periods[0,i] = Returns[i*frequencies_number_of_samples[frequency_index]:\
                                    (i+2)*frequencies_number_of_samples[frequency_index]].flatten()
#     plt.plot(study_periods[0,i])
#     plt.show()
    study_periods[1,i] = dates.iloc[i*frequencies_number_of_samples[frequency_index]:\
                                    (i+2)*frequencies_number_of_samples[frequency_index]].to_numpy().flatten()

In [4]:
number_of_random_search = 1
# print('LSTM')
# LSTM_names, LSTM_results = train_recurrent_model('LSTM',number_of_study_periods,study_periods,number_of_random_search)
# print('GRU')
# GRU_names, GRU_results = train_recurrent_model('GRU',number_of_study_periods,study_periods,number_of_random_search)

print('ARMA')
mse_ARMA = train_ARMA(number_of_study_periods,study_periods)
print(mse_ARMA)

ARMA
Period: 0


c:\program files\python37\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


KeyboardInterrupt: 

In [ ]:
import statsmodels.api as sm
i = 5
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(study_periods[0,i], lags=40, ax=ax1) # 
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(study_periods[0,i], lags=40, ax=ax2)# , lags=40

In [ ]:
print(LSTM_names, LSTM_results)

In [ ]:
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
 
def predict(coef, history):
	yhat = 0.0
	for i in range(1, len(coef)+1):
		yhat += coef[i-1] * history[-i]
	return yhat
 

X = Returns
size = len(X) - 7
train, test = X[0:size], X[size:]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
	model = ARIMA(history, order=(1,0,1))
	model_fit = model.fit(disp=False)
	ar_coef, ma_coef = model_fit.arparams, model_fit.maparams
	resid = model_fit.resid
	yhat = predict(ar_coef, history) + predict(ma_coef, resid)
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('>predicted=%.3f, expected=%.3f' % (yhat, obs))
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)